# The Alcubierre Warp Drive

This notebook explores the Alcubierre warp drive metric in detail.

**Reference**: Alcubierre, M. (1994). Class. Quantum Grav. 11, L73.

## The Metric

The Alcubierre metric in ADM form:

$$ds^2 = -dt^2 + (dx - v_s f(r_s) dt)^2 + dy^2 + dz^2$$

where:
- $v_s$ is the bubble velocity
- $x_s(t) = x_0 + v_s t$ is the bubble center
- $r_s = \sqrt{(x-x_s)^2 + y^2 + z^2}$ is the distance from the bubble center
- $f(r_s)$ is the shape function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from warpbubblesim.metrics import AlcubierreMetric
from warpbubblesim.gr import compute_christoffel, compute_riemann, compute_einstein
from warpbubblesim.gr import compute_energy_density, check_energy_conditions
from warpbubblesim.gr.geodesics import integrate_geodesic
from warpbubblesim.gr.adm import compute_extrinsic_curvature, compute_expansion_scalar
from warpbubblesim.viz.fields2d import plot_energy_density, plot_expansion_scalar, plot_grid_distortion
from warpbubblesim.viz.spacetime_diagrams import plot_spacetime_diagram, plot_light_cones

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

## 1. Shape Functions

The shape function determines how the warp bubble transitions from the interior (flat spacetime) to the exterior (flat spacetime). Several options are implemented:

In [ ]:
# Compare different shape functions
shapes = ['tanh', 'gaussian', 'polynomial', 'smoothstep']
r = np.linspace(0, 3, 200)

plt.figure(figsize=(10, 6))
for shape in shapes:
    metric = AlcubierreMetric(v0=1.0, R=1.0, sigma=8.0, shape=shape)
    f = [metric.shape_function(ri) for ri in r]
    plt.plot(r, f, label=shape, linewidth=2)

plt.xlabel('$r_s$')
plt.ylabel('$f(r_s)$')
plt.title('Comparison of Shape Functions')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=1.0, color='gray', linestyle='--', alpha=0.5)
plt.show()

### Alcubierre's Original (tanh)

$$f(r) = \frac{\tanh(\sigma(r+R)) - \tanh(\sigma(r-R))}{2\tanh(\sigma R)}$$

### Gaussian

$$f(r) = \exp\left(-\frac{r^2}{2R^2/\sigma^2}\right)$$

### Compact Polynomial (C²)

$$f(r) = \begin{cases}(1 - (r/R_{eff})^2)^3 & r < R_{eff}\\ 0 & r \geq R_{eff}\end{cases}$$

## 2. Effect of Parameters

### Velocity $v_s$

In [ ]:
# Different velocities
velocities = [0.1, 0.5, 1.0, 2.0]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for ax, v in zip(axes, velocities):
    metric = AlcubierreMetric(v0=v)
    plot_energy_density(metric, x_range=(-3, 3), y_range=(-3, 3), 
                        nx=80, ny=80, ax=ax)
    ax.set_title(f'$v_s = {v}c$')

plt.suptitle('Energy Density vs Velocity', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Energy density scales as v^2
print("Peak energy density vs velocity:")
for v in [0.1, 0.5, 1.0, 2.0, 5.0]:
    metric = AlcubierreMetric(v0=v)
    # Sample in the wall region
    rho = metric.eulerian_energy_density_analytic(0, 1.0, 0.3, 0)
    print(f"  v = {v}c: rho = {rho:.4e} (ratio to v=1: {rho / -0.0089:.2f})")

### Wall Steepness $\sigma$

In [ ]:
sigmas = [2, 4, 8, 16]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for ax, sigma in zip(axes, sigmas):
    metric = AlcubierreMetric(v0=1.0, sigma=sigma)
    plot_energy_density(metric, x_range=(-3, 3), y_range=(-3, 3), 
                        nx=80, ny=80, ax=ax)
    ax.set_title(f'$\\sigma = {sigma}$')

plt.suptitle('Energy Density vs Wall Steepness', fontsize=14)
plt.tight_layout()
plt.show()

## 3. Energy Density

The energy density measured by Eulerian observers:

$$\rho = -\frac{v_s^2}{32\pi}\left(\frac{df}{dr_s}\right)^2\frac{y^2+z^2}{r_s^2}$$

Key observations:
- **Always negative** in the bubble wall (requires exotic matter)
- Zero at the bubble center and far away
- Zero on the x-axis ($y=z=0$)
- Scales as $v_s^2$

In [ ]:
metric = AlcubierreMetric(v0=1.0)

# Energy density along different cuts
x = np.linspace(-3, 3, 200)

plt.figure(figsize=(10, 6))

for y_val in [0, 0.3, 0.5, 1.0]:
    rho = [metric.eulerian_energy_density_analytic(0, xi, y_val, 0) for xi in x]
    plt.plot(x, rho, label=f'y = {y_val}', linewidth=2)

plt.xlabel('x')
plt.ylabel(r'$\rho$ (energy density)')
plt.title('Energy Density Along x-axis at Different y Values')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='gray', linestyle='--')
plt.show()

## 4. Expansion and Contraction

The expansion scalar $\theta$ shows where space is stretching or compressing:

$$\theta = v_s\frac{x-x_s}{r_s}\frac{df}{dr_s}$$

In [ ]:
metric = AlcubierreMetric(v0=1.0)

fig, ax = plot_expansion_scalar(metric, x_range=(-3, 3), y_range=(-3, 3),
                                 nx=100, ny=100, t=0.0)
plt.title('Expansion Scalar\n(Red=expansion behind, Blue=contraction in front)')
plt.show()

## 5. Grid Distortion

Visualize how the warp bubble distorts the coordinate grid:

In [ ]:
metric = AlcubierreMetric(v0=1.0)

fig, ax = plot_grid_distortion(metric, x_range=(-3, 3), y_range=(-3, 3),
                                n_lines=15, t=0.0)
plt.title('Coordinate Grid Distortion')
plt.show()

## 6. Christoffel Symbols

The Christoffel symbols encode the gravitational field:

In [ ]:
metric = AlcubierreMetric(v0=1.0)
metric_func = metric.get_metric_func()

# Compute Christoffel at a point in the wall
coords = np.array([0, 1.0, 0.3, 0])
gamma = compute_christoffel(metric_func, coords)

print("Non-zero Christoffel symbols at (t=0, x=1, y=0.3, z=0):")
threshold = 1e-10
for mu in range(4):
    for alpha in range(4):
        for beta in range(alpha, 4):  # Symmetry
            if abs(gamma[mu, alpha, beta]) > threshold:
                indices = ['t', 'x', 'y', 'z']
                print(f"  Gamma^{indices[mu]}_{{{indices[alpha]}{indices[beta]}}} = {gamma[mu, alpha, beta]:.6f}")

## 7. Geodesics Through the Bubble

Test particles follow geodesics in the warp bubble spacetime:

In [ ]:
metric = AlcubierreMetric(v0=1.0)
metric_func = metric.get_metric_func()

# Integrate geodesics from different starting positions
results = []
colors = plt.cm.viridis(np.linspace(0, 1, 9))

for i, x0 in enumerate(np.linspace(-4, 4, 9)):
    initial_coords = np.array([0.0, x0, 0.0, 0.0])
    initial_velocity = np.array([1.0, 0.0, 0.0, 0.0])
    
    result = integrate_geodesic(
        metric_func, initial_coords, initial_velocity,
        lambda_span=(0, 15), max_step=0.1
    )
    results.append((result, colors[i]))

plt.figure(figsize=(12, 8))

for result, color in results:
    coords = result['coords']
    plt.plot(coords[:, 1], coords[:, 0], color=color, alpha=0.8)

# Bubble trajectory
t_range = np.linspace(0, 15, 100)
x_bubble = [metric.bubble_center(t) for t in t_range]
plt.plot(x_bubble, t_range, 'k--', linewidth=2, label='Bubble center')

# Bubble edges
plt.fill_betweenx(t_range, 
                   [xb - metric.R for xb in x_bubble],
                   [xb + metric.R for xb in x_bubble],
                   alpha=0.1, color='blue', label='Bubble region')

plt.xlabel('x')
plt.ylabel('t')
plt.title('Geodesics in Alcubierre Spacetime')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(-10, 20)
plt.show()

## 8. Light Cones

The warp bubble distorts light cones, allowing apparent superluminal travel:

In [ ]:
metric = AlcubierreMetric(v0=1.5)  # Superluminal

# Light cone positions
points = [(0, -2), (0, 0), (0, 2),
          (3, -2), (3, 1.5), (3, 5),
          (6, -2), (6, 3), (6, 8)]

fig, ax = plot_light_cones(metric, points, cone_size=0.8,
                           t_range=(0, 10), x_range=(-5, 15))

# Add bubble trajectory
t_range = np.linspace(0, 10, 100)
x_bubble = [metric.bubble_center(t) for t in t_range]
ax.plot(x_bubble, t_range, 'b--', linewidth=2, label='Bubble (v=1.5c)')
ax.legend()
plt.title('Light Cones in Alcubierre Spacetime')
plt.show()

## 9. Energy Conditions

The Alcubierre metric violates all classical energy conditions in the bubble wall:

In [ ]:
metric = AlcubierreMetric(v0=1.0)
metric_func = metric.get_metric_func()

# Check conditions at various points
test_points = [
    ('Center', [0, 0, 0, 0]),
    ('Wall (y=0.3)', [0, 1.0, 0.3, 0]),
    ('Wall (y=0.5)', [0, 1.0, 0.5, 0]),
    ('On axis', [0, 1.0, 0, 0]),
    ('Far away', [0, 5.0, 0, 0]),
]

print("Energy Condition Analysis:")
print("=" * 60)

for name, coords in test_points:
    coords = np.array(coords, dtype=float)
    conditions = check_energy_conditions(metric_func, coords)
    
    print(f"\n{name} {coords}:")
    for cond_name, (satisfied, value) in conditions.items():
        status = "OK" if satisfied else "VIOLATED"
        print(f"  {cond_name}: {status:8s} (value: {value:+.4e})")

## 10. Total Energy Estimate

The total negative energy required is enormous:

In [ ]:
print("Total energy estimates for different velocities:")
print("="*50)

for v in [0.1, 0.5, 1.0, 2.0]:
    metric = AlcubierreMetric(v0=v)
    E = metric.total_energy_estimate()
    
    # Convert to solar masses (in geometric units, M_sun ~ 1.5 km)
    M_sun_km = 1.477  # GM_sun/c^2 in km
    E_solar = E / M_sun_km  # Very rough
    
    print(f"v = {v}c: E ~ {E:.2e} (geometric) ~ {E_solar:.2e} M_sun equivalent")

## 11. Minkowski Limit

Verify that with $v=0$, we recover flat Minkowski spacetime:

In [ ]:
# Minkowski metric: diag(-1, 1, 1, 1)
minkowski = np.diag([-1, 1, 1, 1])

metric = AlcubierreMetric(v0=0.0)  # Zero velocity

# Test at various points
test_points = [
    [0, 0, 0, 0],
    [1, 2, 3, 4],
    [0, 1, 0.5, 0.2],
]

print("Verification that v=0 gives Minkowski:")
for coords in test_points:
    g = metric.metric(*coords)
    diff = np.max(np.abs(g - minkowski))
    print(f"  {coords}: max deviation = {diff:.2e}")

print("\nMetric at (0,0,0,0):")
print(metric.metric(0, 0, 0, 0))

## Summary

The Alcubierre warp drive:

1. **Requires exotic matter** (negative energy density) in the bubble wall
2. **Violates all classical energy conditions** (WEC, NEC, SEC, DEC)
3. **Energy scales as $v^2$** - superluminal velocities require enormous energy
4. **Interior is flat** - passengers experience no forces
5. **Expansion behind, contraction in front** - characteristic signature

This makes it a useful mathematical model for studying exotic spacetimes, even if physical realization remains speculative.